# Training eines NMÜ-Modells mithilfe von Joey NMT

## 1. Installation der benötigten Python-Pakete

In [ ]:
!pip install wheel
!pip install joeynmt
!pip install subword-nmt

# 2. Vorbereitung des Korpus
## 2.1 Download und Vorbereitung der Daten

In [ ]:
# Download eines Shell-Skripts, welches die Daten der IWSLT14-Konferenz herunterlädt und vorbehandelt: (inkl. Aufteilung in Datensätze und BPE)
!curl https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/get_iwslt14_bpe.sh > get_iwslt14_bpe.sh

# Rechte zuteilen, damit Skript ausgeführt werden kann
!chmod +x get_iwslt14_bpe.sh

# Ausführen des Skripts
!./get_iwslt14_bpe.sh

# Verschieben der Daten in das aktuelle Arbeitsverzeichnis
!mv ../test/ .

## 2.2 Vokabular bauen

In [ ]:
# Download des benötigten Skripts
!curl https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py > build_vocab.py

# Rechte zuteilen, damit Skript ausgeführt werden kann
!chmod +x build_vocab.py

# Bau des Vokabulars = Ausführen des Skripts
!./build_vocab.py test/data/iwslt14/train.bpe.32000.de test/data/iwslt14/train.bpe.32000.en

# Vokabular wird in den Ordner mit den Korpusdaten verschoben
!mv vocab.txt test/data/iwslt14/

# 3. Konfiguration des Modells

In [ ]:
# Ordner für Konfiguration erstellen
!mkdir configs

# Download der Standard-Konfiguration
!curl https://raw.githubusercontent.com/joeynmt/joeynmt/master/configs/transformer_iwslt14_deen_bpe.yaml > configs/transformer_iwslt14_deen_bpe.yaml

# 4. Training des Modells

In [ ]:
import yaml
import torch

# Die Konfiguration wird geladen
with open("configs/transformer_iwslt14_deen_bpe.yaml", 'r') as file:
  config = yaml.safe_load(file)

# Gibt es eine CUDA-fähige GPU und es steht in der Konfiguration, dass sie verwendet werden soll, dann wird das Training auf der GPU gestartet

if config["training"]["use_cuda"] == True and torch.cuda.is_available() == True:
  print("--- Training läuft auf der GPU ---")

  !python3 -m joeynmt train configs/transformer_iwslt14_deen_bpe.yaml

# Gibt es keine GPU, wird die Konfiguration geändert und das Training auf der CPU gestartet
else:
  config["training"]["use_cuda"] = False

  with open("configs/transformer_iwslt14_deen_bpe.yaml", 'w') as file:
    yaml.dump(config, file)

  print("--- Training läuft auf der CPU ---")

  !python3 -m joeynmt train configs/transformer_iwslt14_deen_bpe.yaml


# 5. Testen des Modells

In [ ]:
# Starten des Tests, vorhergesagte Übersetzungen werden als "predictions" in das Modellverzeichnis geschrieben
!python3 -m joeynmt test configs/transformer_iwslt14_deen_bpe.yaml --output_path models/iwslt14_deen_bpe_transformer/predictions